In [4]:
from pymongo import MongoClient
import os
import bson

def mongoCollection(connstr, db, collection):
    client = MongoClient(connstr)
    db = client[db]
    return db[collection]

import re
import datetime

collection = mongoCollection(os.environ.get('MONGODB_CONN_STR'), 'smsinfo', 'transactions')

from IPython.display import HTML, display
import tabulate

def extract_transactions(i):
    keys = ['expense_amount', 'payment_mode', 'merchant', 'datetime', 'category', 'sub_category']
#     return list(i[1]['transaction'].values()) + [str(i[1]['_id'])]
    return [i[1]['transaction'].get(key, None) for key in keys] + [str(i[1]['_id'])]

def mark_duplicate(collection, _id):
    result = collection.update_one({'_id': bson.ObjectId(_id)}, {'$set' : {'status': {'duplicate': True}}})
    print('success' if result.modified_count == 1 else 'failed')
    return result

def add_category_info(collection, _id, category='', sub_category=''):
    result = collection.update_one(
        {'_id': bson.ObjectId(_id)}, 
        {'$set' : {'transaction.category': category, 'transaction.sub_category': sub_category}})
    print('success' if result.modified_count == 1 else 'failed')
    return result

In [5]:
year = 2020
month = 2

next_month = 1 if month == 12 else month + 1
next_year = year + 1 if month == 12 else year

monthly = list(enumerate(collection.find({'status': {'analysis_done': True},
                 'transaction.datetime' : {
                     '$gte': datetime.datetime(year,month,1), '$lt': datetime.datetime(next_year,next_month,1)}
                  }, {'transaction': 1})))

monthly_transactions = list(map(extract_transactions, monthly))

In [6]:
display(HTML(tabulate.tabulate(monthly_transactions, tablefmt='html')))

500,CREDIT Card xx3690,PAYTM,2020-02-02 20:44:58,,,5e3a783285ce1ed04ce1b6c3
465.5,CREDIT Card xx3690,FRESHTOH3631510,2020-02-02 20:42:47,,,5e3a783285ce1ed04ce1b6c4
465.5,CREDIT Card xx3690,FRESHTOH3631510 in BANGALORE,2020-02-02 20:42:48,,,5e3a783285ce1ed04ce1b6c5
400,CREDIT Card xx3690,PAYTM3852398,2020-02-02 20:49:59,,,5e3a783285ce1ed04ce1b6c6
199,Debit Card xx1827,NETFLIX ENTERTAINMENT,2020-02-03 19:28:14,,,5e3a783385ce1ed04ce1b6ca
500,CREDIT Card xx3690,PAYTM3852398,2020-02-03 23:14:18,,,5e3a783385ce1ed04ce1b6cb
60,XXX983,bharatpe90200570491@yesbankltd,2020-02-04 00:00:00,,,5e3a783385ce1ed04ce1b6cc
60,XXX983,bharatpe90200570491@yesbankltd,2020-02-06 00:00:00,,,5e3e5b35dd2322b0ad5cccf7
1514.37,CREDIT Card xx3690,www.bigbasket.,2020-02-06 22:08:21,,,5e3e5b35dd2322b0ad5cccfa
588,CREDIT Card xx3690,MYNTRA72883,2020-02-06 22:50:28,,,5e3e5b35dd2322b0ad5cccfc
606,CREDIT Card xx3690,FRESHTOH3631510,2020-02-07 10:39:43,,,5e3e5b35dd2322b0ad5cccff


#feb-2020 message.date > 602238600089916032

`add_category_info(collection, '5e497fb75bc90c80c1c82a81', category='purchase', sub_category='toys')`